## Nimeüksused lausetes test50

In [1]:
from estnltk import Text
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from collections import Counter
import itertools
import csv
from copy import deepcopy

Tühi järjend lausete jaoks:

In [2]:
corpus = []

Tühi järjend lausete ID-de jaoks:

In [3]:
sentence_ids = []

Andmebaasist loetakse sisse laused:

In [4]:
con = sqlite3.connect("media_data_2.db")
cur = con.cursor()

In [5]:
for row in cur.execute("SELECT ID, sentence FROM sentences WHERE named_entity = 1"):
    sentence_id = row[0]
    sentence = json_to_text(json_text=row[1])
    corpus.append(sentence)
    sentence_ids.append(sentence_id)

In [6]:
con.close()

In [7]:
len(corpus)

4492

Vaatame üht lauset ja selle nimeüksusi:

In [11]:
for ner in corpus[1000].ner:
    print(ner.nertag, ner.text, ner.enclosing_text)

PER ['Kuperjanovi'] Kuperjanovi
ORG ['Kuperjanovi', 'pataljoni'] Kuperjanovi pataljoni
LOC ['Mattias'] Mattias
PER ['Konstantin'] Konstantin


NerTagger võib olla teinud vigu, näiteks märkinud isikunime asukohaks, asukoha organisatsiooniks jne. Siin katsetan lihtsakoelist lahendust, mis ei korrigeeri kindlasti kõiki selliseid vigu, kuid võib nende vigade hulka vähendada. Nimelt loetakse kokku eri nimeüksuste esinemised kõigis liikides. Kui nimeüksus esineb rohkem kui ühes liigis, loetakse õigeks liigiks see, milles tema esinemissagedus kõige suurem on. Kui esinemissagedused on eri liikides võrdsed, ei tehta aga praeguses etapis midagi.

Esiteks loon tühjad järjendid nii isikunimede, organisatsiooninimede kui asukohanimede jaoks:

In [12]:
entities_PER = []
entities_ORG = []
entities_LOC = []

Seejärel käin läbi kõik korpuses olevad laused ja lisan nimeüksused järjenditesse vastavalt nende liigile:

In [13]:
for sentence in corpus:
    for ner in sentence.ner:
        lemmatized_ner = [word.lemma[0] for word in ner]
        if ner.nertag == 'PER':
            entities_PER.append(' '.join(lemmatized_ner))
        elif ner.nertag == 'ORG':
            entities_ORG.append(' '.join(lemmatized_ner))
        else:
            entities_LOC.append(' '.join(lemmatized_ner))

In [14]:
len(entities_PER)

2902

In [15]:
len(entities_ORG)

1863

In [16]:
len(entities_LOC)

3642

Seejärel tuleb tekitada kõigi nimeolemiliikide sagedusloendid:

In [17]:
entities_PER_lower = [name.lower() for name in entities_PER]
entities_ORG_lower = [name.lower() for name in entities_ORG]
entities_LOC_lower = [name.lower() for name in entities_LOC]

In [18]:
frequencies_PER = Counter(entities_PER_lower)
frequencies_ORG = Counter(entities_ORG_lower)
frequencies_LOC = Counter(entities_LOC_lower)

In [19]:
#Kümme sagedasemat isiku-, organisatsiooni- ja asukohanime
print('PER' + '\t' + 'ORG' + '\t' + 'LOC')
for (per, org, loc) in itertools.zip_longest(frequencies_PER.most_common(10), frequencies_ORG.most_common(10), frequencies_LOC.most_common(10)):
    print(per, org, loc)

PER	ORG	LOC
('edgar savisaar', 99) ('keskerakond', 215) ('eesti', 1062)
('priit hõbemägi', 66) ('reformierakond', 197) ('tallinn', 262)
('jüri ratas', 61) ('riigikogu', 156) ('euroopa', 156)
('ainar ruussaar', 38) ('irl', 102) ('tartu', 104)
('siim kallas', 37) ('nato', 35) ('venemaa', 101)
('savisaar', 33) ('euroopa liit', 33) ('soome', 80)
('indrek', 32) ('postimees', 29) ('euroopa liit', 78)
('mart luik', 30) ('keskpäevatund', 24) ('eesti vabariik', 73)
('andrus ansip', 27) ('eesti pank', 22) ('läti', 66)
('ansip', 27) ('eas', 22) ('hiina', 57)


Loon uued tühjad järjendid isiku-, organisatsiooni- ja asukohanimede jaoks, mida võib kasutada nimede mugavaks andmebaasi lisamiseks.

In [20]:
entities_PER_new = []
entities_ORG_new = []
entities_LOC_new = []

In [21]:
frequencies_PER_copy = deepcopy(frequencies_PER)
frequencies_ORG_copy = deepcopy(frequencies_ORG)
frequencies_LOC_copy = deepcopy(frequencies_LOC)

In [22]:
equal_occurrences = []

In [23]:
#Kas isikunimede hulgas olevad nimed leiduvad ka organisatsiooni- või asukohanimede hulgas:
for name in frequencies_PER_copy.keys():
    if frequencies_ORG_copy[name] > 0:
        if frequencies_ORG_copy[name] > frequencies_PER_copy[name]:
            print(name, 'ORG:', frequencies_ORG_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_PER[name]
        elif frequencies_ORG_copy[name] < frequencies_PER_copy[name]:
            print(name, 'ORG:', frequencies_ORG_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_ORG[name]
        else:
            equal_occurrences.append(name)
                
    if frequencies_LOC_copy[name] > 0:
        if frequencies_LOC_copy[name] > frequencies_PER_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_PER[name]
        elif frequencies_LOC_copy[name] < frequencies_PER_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_LOC[name]
        else:
            equal_occurrences.append(name)
            
#Kas organisatsiooninimede hulgas leidub nimesid, mis esinevad ka asukohanimede hulgas (isikunimesid pole vaja uuesti läbi käia):
for name in frequencies_ORG_copy.keys():
    if frequencies_LOC_copy[name] > 0:
        if frequencies_LOC_copy[name] > frequencies_ORG_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'ORG:', frequencies_ORG_copy[name])
            del frequencies_ORG[name]
        elif frequencies_LOC_copy[name] < frequencies_ORG_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'ORG:', frequencies_ORG_copy[name])
            del frequencies_LOC[name]
        else:
            equal_occurrences.append(name)
            
#Asukohanimesid pole vaja eraldi läbi käia, seal leiduvad duplikaadid kas isiku- või organisatsiooninimedest on juba tuvastatud.  

taavi LOC: 1 PER: 8
tootsi LOC: 2 PER: 3
rein ORG: 16 PER: 15
rein LOC: 1 PER: 15
priidu ORG: 1 PER: 5
maarja LOC: 1 PER: 2
liis LOC: 1 PER: 2
reinsalu LOC: 11 PER: 15
merkel ORG: 1 PER: 3
merkel LOC: 1 PER: 3
aaviksoo ORG: 1 PER: 6
petseri LOC: 2 PER: 1
ken-mart LOC: 2 PER: 1
reiljan LOC: 1 PER: 6
annika LOC: 1 PER: 2
liivi LOC: 2 PER: 1
facebook ORG: 2 PER: 6
facebook LOC: 15 PER: 6
idea ORG: 1 PER: 2
ojuland ORG: 2 PER: 4
ojuland LOC: 5 PER: 4
samost ORG: 1 PER: 2
ülle madis ORG: 3 PER: 9
teder LOC: 4 PER: 2
tsipras LOC: 1 PER: 3
yana toom ORG: 2 PER: 5
valdo randpere ORG: 1 PER: 2
taavi aas ORG: 1 PER: 8
padar LOC: 2 PER: 1
google ORG: 3 PER: 1
google LOC: 3 PER: 1
nilsson ORG: 1 PER: 3
ratas LOC: 1 PER: 4
palo LOC: 1 PER: 2
viimsi LOC: 2 PER: 1
madis kimmel ORG: 3 PER: 15
vabadus ORG: 4 PER: 2
forbes ORG: 1 PER: 2
eston kohver ORG: 1 PER: 4
laaneots LOC: 1 PER: 2
kunnas LOC: 1 PER: 2
brexit LOC: 3 PER: 1
ekre ORG: 1 PER: 9
ekre LOC: 7 PER: 9
mart helme ORG: 1 PER: 8
danske ORG: 4 

In [24]:
equal_occurrences = set(equal_occurrences)
print(equal_occurrences)

{'baltic', 'allikmaa', 'midfield', 'infortar', 'stasi', 'guardian', 'parts', 'pihl', 'külliki kübarsepp', 'times', 'selver', 'tehnikaülikool', 'helme', 'elva', 'aidla', 'brexit', 'hekre', 'vabariik', 'lepik', 'merkel', 'estrada', 'eiffel', 'riia riia', 'toots', 'ossinovsk', 'enef', 'fifa', 'taska', 'leedo', 'magnitsk', 'google', 'väino linde', 'ak', 'süvari', 'alex', 'liina', 'kaubandus-tööstuskoda', 'päevaleht'}


In [25]:
for name in frequencies_PER.keys():
    entities_PER_new.append(name)
    
for name in frequencies_ORG.keys():
    entities_ORG_new.append(name)
    
for name in frequencies_LOC.keys():
    entities_LOC_new.append(name)

Saadud parandustega sagedusloendid kirjutatakse talletatakse csv-failis, et võimaldada vajadusel käsitsi korrektuuride tegemist.

In [27]:
with open('ner_frequencies.csv', 'w', newline='', encoding='UTF-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['name', 'freq', 'value'])
    writer.writeheader()
    header2 = {'name': 'string', 'freq': 'integer', 'value': 'string'}
    writer.writerow(header2)
    for name in frequencies_PER.keys():
        writer.writerow({'name': name, 'freq': frequencies_PER[name], 'value': 'PER'})
    for name in frequencies_ORG.keys():
        writer.writerow({'name': name, 'freq': frequencies_ORG[name], 'value': 'ORG'})
    for name in frequencies_LOC.keys():
        writer.writerow({'name': name, 'freq': frequencies_LOC[name], 'value': 'LOC'})

In [28]:
with open('ner_frequencies.csv', encoding='UTF-8') as csv_file:
    rows = []
    reader = csv.DictReader(csv_file)
    for row in reader:
        rows.append(row)

In [29]:
#Esimesed 10 rida
print(rows[:10])

[{'name': 'string', 'freq': 'integer', 'value': 'string'}, {'name': 'hans', 'freq': '5', 'value': 'PER'}, {'name': 'eerik', 'freq': '2', 'value': 'PER'}, {'name': 'andrus ansip', 'freq': '27', 'value': 'PER'}, {'name': 'meelis', 'freq': '8', 'value': 'PER'}, {'name': 'olev esula', 'freq': '1', 'value': 'PER'}, {'name': 'vadim', 'freq': '2', 'value': 'PER'}, {'name': 'juskin', 'freq': '1', 'value': 'PER'}, {'name': 'margus', 'freq': '2', 'value': 'PER'}, {'name': 'sven', 'freq': '9', 'value': 'PER'}]


Järgnevalt luuakse andmebaasi uus tabel huvipakkuvate nimeüksuste hoiustamiseks (isiku- ja organisatsiooninimed).

In [33]:
con = sqlite3.connect("media_data_2.db")
cur = con.cursor()

In [34]:
cur.execute("CREATE TABLE named_entities(ID INTEGER PRIMARY KEY, sentence_ID INTEGER, PER TEXT, ORG TEXT)")

In [35]:
for i in range(len(corpus)):
    for ner in corpus[i].ner:
        lemmatized_ner = [word.lemma[0] for word in ner]
        lemmatized_ner_text = ' '.join(lemmatized_ner).lower()
        if ner.nertag == 'PER' and lemmatized_ner_text in entities_PER_new:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, PER)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag != 'PER' and lemmatized_ner_text in entities_PER_new and lemmatized_ner_text not in equal_occurrences:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, PER)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag == 'ORG' and lemmatized_ner_text in entities_ORG_new:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, ORG)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag != 'ORG' and lemmatized_ner_text in entities_ORG_new and lemmatized_ner_text not in equal_occurrences:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, ORG)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        con.commit()

In [36]:
con.close()

Praeguse seisuga ei püüta parandada vigaseks hinnatud ner-olemeid, vaid luua ainult parandatud kujul nimekiri ner-olemitest